# 11: Filters

Today I want to define a filter as a function (or chain of functions) that can be applied to a series of inputs, something like:

$$
\mathrm{Inputs} \rightarrow \mathrm{Filter} \leftrightarrow \mathrm{Outputs}
$$

Let's say that you have a filter defined for video chat on your phone, which draws dog ears on your head, that could be implemented without any feedback:


$$
\mathrm{Frame\,In,}\,f_i \rightarrow [x_i=\mathrm{findHead}(f_i);\,g_i=\mathrm{addEars}(x_i,f_i)] \rightarrow \mathrm{Frame\, Out,}\,g_i
$$

A smart programmer will recognise that typical video feeds have continuity, so the dog ears shouldn't just jump around randomly in the picture.  We can make use of the assumption of continuity to design a better dog ears filter, like this:

$$
\mathrm{Frame\,In,}\,f_i \rightarrow [x_i=\mathrm{findHead}(f_i,x_{i-1},x_{i-2});\,g_i=\mathrm{addEars}(x_i,f_i)] \leftrightarrow \mathrm{Frame\, Out,}\,g_i
$$

I am going to call a filter with feedback, which has some memory, or some internal state, a 'Finite State Machine' (FSM), this is just a piece of computer science jargon that you might as well know.   *The* FSM (the "Flying Spaghetti Monster") is not related to this section of the course.







## MC Stepsize

The Monte-Carlo code that we implemented for exercise 09 used totally arbitrary re-positioning of particles. This is OK in a low density system, but will become inefficient in a high density system as the majority of proposed moves will lead to an overlap.   

This week I'd like you to re-implement the MC code (if you didn't get 09 working, then start by doing that, before you do this exercise).

With a working MC code, I'm going to ask you in the Assignment section to implement a filter to discover the optimum stepsize.  I'll give you the function to propose a random MC move of a given size $l_{step}$ directly:



In [1]:
#include <math.h>
#include <stdio.h>     
#include <stdlib.h>

void propose_move( double *old_x, double *new_x, double l_step, double box_L ){
    /* function to generate a random vector of length l_step, 
       and add it to old_x[], writing the output to new_x[] (after imaging)
       */
    double   rana, ranb, ransq, factor;
    double   half_L;
    double   two_by_rand_max = 2.0 / RAND_MAX;
    int      d;
    
    //It turns out that generating a random point in a sphere with an 
    //isotropic distribution is quite tricky, at least it is tricky to do 
    //this efficiently.
    ransq = 99.9;
    while(ransq >= 1) {
        rana  = 1 - ( two_by_rand_max * rand() ); //random number on [-1..1)
        ranb  = 1 - ( two_by_rand_max * rand() ); //random number on [-1..1)
        ransq = rana*rana + ranb*ranb;            //squared length, on [0..sqrt(2))
    }//while loop: if random 2D point was outside the unit circle, we try again.
    
    //if 2D point was inside the unit circle, we can define the third component from the other two.
    factor = 2*l_step*sqrt(1-ransq);
    
    //use trigonometry to define x,y,z coordinates for a vector vased on two random numbers.
    new_x[0] = old_x[0] + rana*factor;  
    new_x[1] = old_x[1] + ranb*factor;
    new_x[2] = old_x[2] + l_step-2*factor*ransq;

    //alternative algorithm:
    //  1) generate a point in the unit cube
    //  2) if not in unit sphere, back to 1.
    // ... the method I provided here only makes two calls to the RNG, so it is more efficient,
    // although harder to understand without a diagram.
    // 
    // rejection rate for cube->sphere   = 1 - (1.333 * pi * 0.5^3) = ~48%
    // rejection rate for square->circle = 1 - (pi * 0.5 ^2) = ~21%
    
    //image the new point
    half_L = 0.5 * box_L;
    for(d = 0; d < 3; d++){
        while( new_x[d] >= half_L ) new_x[d] -= box_L;
        while( new_x[d] < -half_L ) new_x[d] += box_L;
    }
}



In [2]:
#include <stdio.h>
#include <stdlib.h>

//define an int array with information that
//we can use to change its size if it gets too big.
typedef struct xints_t {
    int *data;            //pointer to the stored data
    int  n_data_items;    //number of items in the array
    int  size_in_mem;     //size in memory (can be bigger but not smaller than n_data_items)
} XINTS_T;

//Define a (listable) structured type ATOM_T to hold info about a (classical) atom
//I've simplified the datatype to only contain info that is needed for today's problem
typedef struct atom_t {
    atom_t   *listNext;      //can add to a list of similar objects
    XINTS_T  *nebIds;        //keep an extendable array of int indices of neighbours
    int       myId;          //what am I?
    double    position[3];   
} ATOM_T; 


In [3]:

XINTS_T *create_xints(){
    XINTS_T *xints;
    
    xints = (XINTS_T *)malloc(sizeof(xints));
    xints->n_data_items = 0;
    xints->size_in_mem  = 8;
    xints->data = (int *)malloc(8*sizeof(int));
    
    return( xints );
}


In [4]:
void free_xints( XINTS_T *xints ){
    free(xints->data);
    free(xints);
}

In [5]:
void append_xint( XINTS_T *xints, int i ){
    if( xints->n_data_items >= xints->size_in_mem ){
        xints->size_in_mem *= 2; //realloc is expensive, so take more than we need.
        xints->data = (int *)realloc(xints->data, xints->size_in_mem*sizeof(int) );
    }
    xints->data[xints->n_data_items] = i; //save the new array member.
    xints->n_data_items             += 1;
}

In [6]:
void delete_xint( XINTS_T *xints, int remove_i ){
    
    //delete first occurence of 'remove_i' from the xints.
    int i, j;
    
    for( i = 0; i < xints->n_data_items; i++ ){
        if( xints->data[i] == remove_i ){
            for( j = i; j < xints->n_data_items - 1; j++ ){
                xints->data[j] = xints->data[j+1];
            }
            xints->n_data_items--;
            break;
        }
    }
}

In [7]:
ATOM_T *newAtom( double x, double y, double z, int id ){
    
    /* create an atom in a given position */
    
    ATOM_T *a;
    
    a = (ATOM_T *)malloc( sizeof(ATOM_T) );
    
    //info about the atom itself
    a->listNext     = NULL;
    a->myId         = id;
    a->position[0]  = x;
    a->position[1]  = y;
    a->position[2]  = z;
    
    //allocate some empty (but extendable) arrays to 
    //store connectivity information.
    a->nebIds       = create_xints();
    
    return( a );
}


In [8]:
void freeAtom( ATOM_T *a ){
    free( a->nebIds->data );
    free( a->nebIds);
    free( a );
}

In [9]:
ATOM_T **boxOfAtoms(int N_atoms, double box_L, int seed ){
    
    /*
     create some atoms and place them randomly in a box centred at the origin.
    */
    ATOM_T **atoms;
    int          i;
    double  half_L;
    
    half_L = box_L * 0.5;
    
    //init the random number generator, so that code is repeatable
    srand( seed );
    
    atoms = (ATOM_T **)malloc(N_atoms*sizeof(ATOM_T *));
    for( i = 0; i < N_atoms; i++ ){
        atoms[i] = newAtom( (rand()*(box_L/RAND_MAX)) - half_L,
                            (rand()*(box_L/RAND_MAX)) - half_L,
                            (rand()*(box_L/RAND_MAX)) - half_L, 
                             i );
        
    }
    return( atoms );
}

In [10]:
int image_int( int i, int N ){
    if( i >= N ) return ( i - N );
    if( i < 0  ) return ( i + N );
    return( i );
}

In [11]:
double check_contact( ATOM_T* a, ATOM_T* b, double box_L, double r_cut ){
    /* 
    check if two atoms are in "contact" (closer than distance r_cut)
    
    subject to periodic boundary conditions
    */
    double dx[3], r2, half_L;
    int    d;
    
    half_L = box_L * 0.5;
    for( d = 0; d < 3; d++ ){
      //displacement vector a to b.
      dx[d] = b->position[d] - a->position[d];
        
      //periodic imaging, now see nearest image.
      if     ( dx[d] >  half_L ) dx[d] -= box_L;
      else if( dx[d] < -half_L ) dx[d] += box_L;
    }
    
    r2 = dx[0]*dx[0] + dx[1]*dx[1] + dx[2]*dx[2]; 
    
    //return r if we are in contact, otherwise -1 (impossible r).
    if( r2 > r_cut*r_cut ) return( -1.0 );
    return( sqrt(r2) );
    
}

In [12]:
double U_LJpair( double *x1,  double *x2, double box_L, double r_cut ){
    /* 
    return Lennard-Jones interaction energy between two points
    */
    double dx[3], r2, inv_r6, inv_rcut6, dU, half_L;
    int    d;
    
    half_L = box_L * 0.5;
    for( d = 0; d < 3; d++ ){
      //displacement vector a to b.
      dx[d] = x2[d] - x1[d];
        
      //periodic imaging, now see nearest image.
      if     ( dx[d] >  half_L ) dx[d] -= box_L;
      else if( dx[d] < -half_L ) dx[d] += box_L;
    }
    
    r2 = dx[0]*dx[0] + dx[1]*dx[1] + dx[2]*dx[2]; 
    
    if( r2 > r_cut*r_cut ) return( 0.0 );
    inv_r6 = 1. / ( r2*r2*r2 );
    
    dU    = inv_r6*inv_r6 - inv_r6;
    
    //JTB lazy coding: just evaluate the shift every time the function is called
    inv_rcut6  = 1./r_cut;
    inv_rcut6 *= inv_rcut6;
    inv_rcut6  = inv_rcut6 * inv_rcut6 * inv_rcut6;
    dU        -= (inv_rcut6*inv_rcut6 - inv_rcut6);
    
    return( dU );
    
}

In [13]:
double U_LJsys( ATOM_T **atoms, int N_atoms, double box_L, double r_cut ){
    /* 
    return total Lennard-Jones interaction energy
    */
    double dx[3], r2, inv_r6, inv_rcut6, dU, half_L;
    int    i, j, n, d;
    double U_LJ = 0.0;
    
    //for each atom
    for( i = 0; i < N_atoms; i++ ){
        
        //for each neighbour of that atom
        for( j = 0; j < atoms[i]->nebIds->n_data_items; j++){
            
            //id of jth neighbour
            n     = atoms[i]->nebIds->data[j];
            
            if( n > i ){          
               U_LJ += U_LJpair( atoms[i]->position,  atoms[n]->position, box_L, r_cut );
            }
        }
    }
    return( U_LJ );
}

In [14]:
void assignCellIndex( double *posn, int *ix, int *iy, int *iz, double box_L, double r_cell ){
    /* assign cell indices for points on [-0.5*box_L.. 0.5*box_L) 
    
       This is done by integer rounding-down, assumes that the 
       box is centred at the origin and that all particles are
       in the box.   [-L/2 <= x,y,z < L/2)
    */
    double half_L;
    
    half_L = box_L * 0.5;
    
    //write to the output variables provided by the pointers ix, iy, iz
   *ix = (int)((posn[0]+half_L)/r_cell);
   *iy = (int)((posn[1]+half_L)/r_cell);
   *iz = (int)((posn[2]+half_L)/r_cell);
}

In [15]:
ATOM_T **buildCellList( ATOM_T **atoms, int N, double L, double rcut ){
    
    ATOM_T **cells;
    
    double r_cell, rcontact;
    int    Ncells_x, Ncells_x2, Ncells, i;
    int    ii, jj, kk;
    
    //how big should the cells be, and how many do we need?
    Ncells_x  = (int)(L / rcut);      //count the number of cells in each direction
    if( Ncells_x < 3 )Ncells_x = 3;   //minimum is three
    r_cell    = L / Ncells_x;         //length per cell
    Ncells_x2 = Ncells_x * Ncells_x;
    Ncells    = Ncells_x * Ncells_x * Ncells_x;
    
    //allocate space for the cell array
    cells = (ATOM_T**)malloc(Ncells*sizeof(ATOM_T*));
    for(i = 0; i < Ncells; i++) cells[i] = NULL; //cells start off empty
    
    //assign atoms to cells
    for(i = 0; i < N; i++){
        //get cell indices ii, jj, kk
        assignCellIndex( atoms[i]->position, &ii, &jj, &kk, L, r_cell );
        
        //add the atom to the cell it belongs to (ii + jj*Ncells_x + kk*Ncells_x2)
        atoms[i]->listNext                 = cells[ii+jj*Ncells_x+kk*Ncells_x2];
        cells[ii+jj*Ncells_x+kk*Ncells_x2] = atoms[i];
        
    }
    return ( cells );
}

In [16]:
void buildNeighbourLists(  ATOM_T **atoms, int N, double L, double rcut, ATOM_T **cells ) {
    
    ATOM_T *a, *b;
    
    //duplicate some convenience variables that depend on rcut and L
    double r_cell, rcontact;
    int    Ncells_x, Ncells_x2, Ncells, i;
    int    ii, jj, kk;
    int    iii, jjj, kkk,  i0, j0, k0;
    
    Ncells_x  = (int)(L / rcut);      //count the number of cells in each direction
    if( Ncells_x < 3 )Ncells_x = 3;   //minimum is three
    r_cell    = L / Ncells_x;         //length per cell
    Ncells_x2 = Ncells_x * Ncells_x;
    Ncells    = Ncells_x * Ncells_x * Ncells_x;
    
    /*
    *  check contacts and build neighbour xarrays
    */
    //loop over all cells.
    for(i = 0; i < Ncells; i++){
        
        //i0,j0,k0 indices of this cell in 3D
        i0 =       i % Ncells_x;
        j0 =      (i % Ncells_x2) / Ncells_x;
        k0 = (int) i / Ncells_x2;
        
        //a is the first atom in the cell, or NULL if no atoms
        a = cells[i];
        while( a != NULL ){
            
            //check all 27 neighbouring cells
            for( ii = -1; ii <= 1; ii++ ){
                iii = image_int( i0+ii, Ncells_x );
            for( jj = -1; jj <= 1; jj++ ){
                jjj = image_int( j0+jj, Ncells_x );
            for( kk = -1; kk <= 1; kk++ ){
                kkk = image_int( k0+kk, Ncells_x );
                b = cells[ iii + jjj*Ncells_x + kkk*Ncells_x2 ];
                if( a == b ) continue;
                while( b ){
                   if( b != a ){
                       if( check_contact(a, b, L, rcut) >= 0 ){
                           
                           //new code for this week: add the atom b
                           //to the neighbour xarray of a.
                           append_xint( a->nebIds, b->myId );
                               
                       }
                   }
                   b = b->listNext;  //keep looking in this neighbour cell
                }
            }
            }
            }
            a = a->listNext; //keep looking in the main cell.
        }
    }
}

In [17]:
double moveParticle(  ATOM_T **atoms, int N, double L, double rcut, ATOM_T **cells, int move_i, double* new_x ) {
    
    ATOM_T *a, *b, *bprev;
    
    //duplicate some convenience variables that depend on rcut and L
    double r_cell, rcontact;
    int    Ncells_x, Ncells_x2, Ncells, i, j, n, already_nebs;
    int    ii, jj, kk;
    int    new_icell, old_icell;
    int    iii, jjj, kkk,  i0, j0, k0;
    
    Ncells_x  = (int)(L / rcut);      //count the number of cells in each direction
    if( Ncells_x < 3 )Ncells_x = 3;   //minimum is three
    r_cell    = L / Ncells_x;         //length per cell
    Ncells_x2 = Ncells_x * Ncells_x;
    Ncells    = Ncells_x * Ncells_x * Ncells_x;
    
    //working variables for the MC move
    double old_Uat, new_Uat, delta_Uat;
    
    //get a pointer to the atom to be moved
    a = atoms[move_i];
    
    //Current neighbourhood energy interactions
    //for each neighbour of that atom
    old_Uat    = 0.0;
    for( j = 0; j < a->nebIds->n_data_items; j++){
        
            //id of jth neighbour
            n = a->nebIds->data[j];
           
            //Add LJ energy contribution.
            old_Uat += U_LJpair( a->position,  atoms[n]->position, L, rcut );
        
            //clean atom a from the neighbour list of b
            delete_xint( atoms[n]->nebIds, move_i );
        
    }
    
    //refresh the neighbour list of a and build a new one for the new position.
    free_xints( a->nebIds );
    a->nebIds = create_xints();
    
    //what cell was atom 'a' in?
    assignCellIndex( a->position, &i0, &j0, &k0, L, r_cell );    
    old_icell = i0 + j0*Ncells_x + k0*Ncells_x2;
    
    //what cell is atom 'a' moving to?
    assignCellIndex( new_x,       &i0, &j0, &k0, L, r_cell );    
    new_icell = i0 + j0*Ncells_x + k0*Ncells_x2;
    
    //restitch the cell pointers
    b     = cells[old_icell];
    bprev = NULL;
    while( b ){ //loop over atoms in the cell
        if( a == b ){ //remove atom a when we find it.
            if( bprev ) bprev->listNext  = a->listNext;
            else        cells[old_icell] = a->listNext;
            break;
        }
        bprev = b;
        b     = b->listNext;
    }
    
    //add atom a to new cell (head of list).
    a->listNext = cells[new_icell];
    cells[new_icell] = a;
    
    //update position
    for(i = 0; i < 3; i++ ) a->position[i] = new_x[i];
     
    //energy calculation: check all 27 neighbouring cells
    new_Uat = 0.0;
    for( ii = -1; ii <= 1; ii++ ){
         iii = image_int( i0+ii, Ncells_x );
         for( jj = -1; jj <= 1; jj++ ){
            jjj = image_int( j0+jj, Ncells_x );
            for( kk = -1; kk <= 1; kk++ ){
                kkk = image_int( k0+kk, Ncells_x );
                    
                //neighbour atom 'b'
                b = cells[ iii + jjj*Ncells_x + kkk*Ncells_x2 ];
                while( b ){
                    
                       //don't compare an atom to itself.
                       if( a == b ){
                           b = b->listNext;
                           continue;
                       }
                        
                       //get the energy change
                       delta_Uat = U_LJpair( a->position,  b->position, L, rcut );
                    
                       rcontact = check_contact( a, b, L, rcut );
                    
                       if( rcontact != 0.0 ){
                           
                           //create the new neighbour list as we go
                           append_xint( a->nebIds, b->myId );
                           
                           //need to check that atom a is not already 
                           //in b's neighbour list before we add it.
                           already_nebs = 0;
                           for( n = 0; n < b->nebIds->n_data_items; n++ ){
                               if( move_i == b->nebIds->data[n] ) {
                                   already_nebs = 1;
                                   break;
                               }
                           }
                           if( already_nebs == 0 )append_xint( b->nebIds, move_i );
                           
                           new_Uat += delta_Uat;
                       }
                       b = b->listNext;  //keep looking in this neighbour cell
                }
            }
        }
    }
    return( new_Uat - old_Uat );
    
}

In [18]:
int test_MC(int N_atoms, double box_L, int seed, double r_cut, int N_sweeps, double start_movesize){
    
    /*
      Test code to:
      
      run a metropolis monte carlo.
      
      **STUB CODE : INCOMPLETE: *YOU* HAVE TO GET IT WORKING.
      
    */
    ATOM_T **atoms, **cells;
    int      i, j, n, d, i_sweep, n_sweeps;
    int      i_atom;
    double   new_x[3], half_L, dU, U_start, U_end, sum_dU;
    
    double   move_size;
    
    half_L = box_L * 0.5;
    
    //place a bunch of atoms randomly in a box of size box_L
    atoms = boxOfAtoms( N_atoms, box_L, seed );
    
    //build a cell list so we can easily find neighbours of atoms
    cells = buildCellList( atoms, N_atoms, box_L, r_cut);
    
    //give each atom a list of its neighbours, stored inside the ATOM_T
    //as an extensible array.
    buildNeighbourLists( atoms, N_atoms, box_L, r_cut, cells );
    
    //for debug / checking:
    U_start = U_LJsys( atoms, N_atoms, box_L, r_cut );
    
    
    move_size = start_movesize;
    
    
    /*******************MMC*/
    sum_dU = 0.0;
    for( i_sweep = 0; i_sweep < N_sweeps; i_sweep++){
        
        //each 'sweep' move each atom once.
        for(i_atom = 0; i_atom < N_atoms; i_atom ++ ){
         
            //pick a new position a fixed distance away
            propose_move( atoms[i_atom]->position, new_x, move_size, box_L );
            
            //what is the energy change?
            dU = moveParticle( atoms, N_atoms, box_L, r_cut, cells, i_atom, new_x );
            sum_dU += dU;
        }
    }
    
    //debug / checking: was there an energy drift?
    U_end = U_LJsys( atoms, N_atoms, box_L, r_cut );
    
    printf("after %i sweeps, U went from %lf to %lf,\n delta should be %lf, delta was: %lf  drift of: %e\n\n",
                N_sweeps, U_start, U_end, U_end - U_start, sum_dU, fabs((U_end - U_start) - sum_dU));
    
    
    /**********************/
    
    //clean up.
    for(i_atom = 0; i_atom < N_atoms; i_atom ++ ) freeAtom( atoms[i_atom] ); //free the atoms one at a time
    free( atoms ); //this releases the array of double-pointers to atoms.
    free( cells ); //a cell is just a pointer to the first atom in a list, we can free directly.
    
    
    return( EXIT_SUCCESS );
}

In [19]:
test_MC( 100, 15.0, 1331, 4.0, 30, 1.0);
test_MC( 100, 15.0, 1332, 4.0, 30, 1.0);
test_MC( 100, 15.0, 1333, 4.0, 30, 1.0);
test_MC( 100, 15.0, 1334, 4.0, 30, 1.0);
test_MC( 100, 15.0, 1335, 4.0, 30, 1.0);

after 30 sweeps, U went from 107.122428 to 2010717894.242830,
 delta should be 2010717787.120403, delta was: 2010717787.120404  drift of: 1.430511e-06

after 30 sweeps, U went from 68.448259 to 3146.093749,
 delta should be 3077.645490, delta was: 3077.645626  drift of: 1.364590e-04

after 30 sweeps, U went from 1448.963768 to 1079677.530124,
 delta should be 1078228.566356, delta was: 1078228.566261  drift of: 9.593298e-05

after 30 sweeps, U went from 199.874158 to 2243.570967,
 delta should be 2043.696809, delta was: 2043.681929  drift of: 1.487947e-02

after 30 sweeps, U went from 24638.663699 to 114567.244251,
 delta should be 89928.580552, delta was: 89928.580555  drift of: 3.446170e-06



OK great I have given you some code to 

1) Create a box of atoms

2) See if two atoms are in contact

3) Make an extensible array of atom ids showing the neighbours of each atom.

4) Displace atoms by a fixed distance in a random direction

5) Track the evolving potential energy for displacements


Before I do the whole exercise I am going to stop coding and ask you to take over.  What is missing is the Metropolis criterion (see exercise 09), and also a filter for setting the step size, which I'm going to explain below.


## Assignment, week 11: Optimum Step Size Filter

In order to have an efficient MC code, moves should be accepted roughly half the time.  If no move is ever accepted, nothing happens.  If all the moves are accepted, then they are probably too small and the system will evolve more slowly than it needs to.  We can define a Finite State Machine to optimise the step size as we continue our simulation, something like:


0) Set $N_{max} >> 0$.

1) Initialise the stepsize $l_{step} \leftarrow 1$, the step counter $n_{step} \leftarrow 0$, and $p\leftarrow 0$

2) Make $N_{atoms}$ moves (a sweep) with size $l_{step}$, and calculate the acceptance rate $x$ over the sweep.

3) Estimate the current mean acceptance rate $p\leftarrow p\frac{n_{step}}{n_{step}+1} + \frac{x}{n_{step}+1}$

4) $l_{step} \leftarrow l_{step} \frac{\gamma + p}{\gamma + 0.5}$

5) iff $n_{step}<N_{max}$: $n_{step} \leftarrow n_{step} + 1$

6) back to (2)


### Filter behaviour:

A short look at the behaviour of this filter shows us that it should have the following properties: 

A) $l_{step}$ always increases when $p > 0.5$, with a maximum ratio $\frac{1+\gamma}{\gamma+0.5}$.

B) $l_{step}$ always decreases when $p < 0.5$, with a minimum ratio $\frac{\gamma}{\gamma+0.5}$.  The factor $\gamma$ is used here just to stabilise the filter.

C) Changes in $p$ gradually become less sensitive with increasing $n_{step}$, so the filter should automatically stabilise (oscillations should die out).  

D) Bad property: information from the first steps is never forgotten, so a dramatically low acceptance rate in the beginning can "poison" the rest of the run.  We control this effect by setting some $N_{max} >> 0$ after which new information stops decreasing in weight (we stop increasing the counter $n_{step}$ for the purposes of the filter).  This has its effect on step *(3)* of the filter algorithm.


### Filters and Detailed Balance:

Recall from the derivation of the Metropolis Algorithm, that thermodynamic equilibrium requires time symmetry in the probability to make any given transition in the system.  Implementing a filter which dynamically sets the move size is obviously a breach of this assumption, so we need to be aware that the simulation has for sure not converged until some time after the filter has stabilised to a consistent stepsize, or, better, just been turned off.


(1) Implement the MMC algorithm (you may cut/paste from previous exercise 09).

(2) Run MMC for multiple temperatures, with adaptive stepsizing for the first $X$ sweeps ($X$).  We should keep the number of particles and the other parameters constant, and report the current stepsize $l_{step}$, as well as potential energy scaled per particle $U/N$.

$$
N = 100
$$

$$
L = 15.0
$$

$$
r_{cut} = 4.0
$$

$$
\mathrm{seed} = 1337
$$

I'm going to ask you to use the "leet" number as a seed so that all code is repeatable.

$k_B T$ = 0.0, 0.5, 1.0, 2.0.  

The goal for this exercise is much the same as for the estimation of $<U/N>_{|T}$ in the previous work, but this time I also want you to find and report the optimum stepsize $l^*_{|T}$ for each temperature.  With adaptive stepsizing the $T=0$ run in particular, I am hoping, should converge more quickly and give a more interesting structure.

As previously I'll give 110% to whoever gets closest to my reference values (that I'll use a proper computer to find, overnight); everyone else who gets the exercise correct and has sane values will get 100%.




In [20]:
void propose_move( double *old_x, double *new_x, double l_step, double box_L )

In [21]:
int MC_LHB(int N_atoms, double box_L, int seed, double r_cut, int N_sweeps, double beta){
    
    /*
      Test code to:
      
      run a metropolis monte carlo.
      
      **STUB CODE : INCOMPLETE: *YOU* HAVE TO GET IT WORKING.
      
    */
    ATOM_T **atoms, **cells;
    int      i, j, n, d, i_sweep, n_sweeps;
    int      i_atom;
    double   *pU;
    double   new_x[3], old_x[3], half_L, dU, U_total, U_start, U_end, sum_dU, U_old, U_new, U_avg, P_accept, deviation;
    
    half_L = box_L * 0.5;
    
    srand( seed );
    
    U_total = 0.;
    U_avg = 0.;
    
    
    // allocate memory for U values
    pU = (double *)malloc(N_sweeps*sizeof(double));
    
    //place a bunch of atoms randomly in a box of size box_L
    atoms = boxOfAtoms( N_atoms, box_L, seed );
    
    //build a cell list so we can easily find neighbours of atoms
    cells = buildCellList( atoms, N_atoms, box_L, r_cut);
    
    //give each atom a list of its neighbours, stored inside the ATOM_T
    //as an extensible array.
    buildNeighbourLists( atoms, N_atoms, box_L, r_cut, cells );
    
    //for debug / checking:
    U_start = U_LJsys( atoms, N_atoms, box_L, r_cut );
    U_old = U_start;
    
    /*******************MMC*/
    sum_dU = 0.0;
    for( i_sweep = 0; i_sweep < N_sweeps; i_sweep++){
        
        //each 'sweep' move each atom once.
        for(i_atom = 0; i_atom < N_atoms; i_atom ++ ){
         
            //pick a random position
            for( d = 0; d < 3; d++ ){
                old_x[d] = atoms[i_atom]->position[d];
                new_x[d] = (rand()*(box_L/RAND_MAX)) - half_L; // insert propose move here + keep counter of success
            }

            //what is the energy change?
            dU = moveParticle( atoms, N_atoms, box_L, r_cut, cells, i_atom, new_x );
                        
            // update new energy value
            U_new = U_old + dU;

            // Metropolis Criterion
            if (dU <= 0){
                pU[i_sweep] += U_new; // store the energy of the run
            }
            else{
                P_accept = (double)rand()/RAND_MAX; // probability between 0 and 1
                if (P_accept > exp(- beta * dU) ){ // not accepted
                    // revert changes made to the system
                    U_new = U_old;
                    pU[i_sweep] += U_new;
                    moveParticle( atoms, N_atoms, box_L, r_cut, cells, i_atom, old_x ); 
                }
                else { pU[i_sweep] += U_new; }
            }
            
            // track energy change
            sum_dU += U_new - U_old;

            // invert assignment for the next loop
            U_old = U_new;  
        }
        // pU[i_sweep] = U_new;
        U_total += pU[i_sweep];
        U_avg = U_total/N_sweeps/N_atoms;
    }
    
    U_end = U_LJsys( atoms, N_atoms, box_L, r_cut );
    deviation = fabs((U_end - U_start) - sum_dU);

    printf("For beta = %f\n\n",beta);
    printf("U went from %lf to %lf after sweeps,\ndelta should be %lf, delta was: %lf, deviation of: %e\n",
                U_start, U_end, U_end - U_start, sum_dU, deviation);
    printf("Total energy = %f \nAverage Energy of the systewm is: %f\n\n--------------------------------\n", U_total, U_avg);
    
    
    /**********************/
    
    //clean up.
    free( atoms ); //this releases the array of double-pointers to atoms.
    free( cells ); //a cell is just a pointer to the first atom in a list, we can free directly.
    
    return( EXIT_SUCCESS );
}

In [22]:
int filter(int N_atoms, double box_L, int seed, double r_cut, double start_movesize, double beta){
    
    // filter settings
    int    N_max  = 25;
    int    n_step = 0;
    double l_step = 1;
    double p      = 0;
    double x      = 0;
    int acc_counter  = 0;
    double gamma  = 0.1;
    
    // atoms settings
    ATOM_T **atoms, **cells;
    int      i, j, n, d, i_sweep, n_sweeps;
    int      i_atom;
    double   old_x[3], new_x[3], half_L, dU, U_start, U_end, sum_dU, P_accept, P_pass, U_old, U_new;
    
    half_L = box_L * 0.5;
    
    double move_size;
    
    move_size = start_movesize;
    
    srand( seed );
    
    //place a bunch of atoms randomly in a box of size box_L
    atoms = boxOfAtoms( N_atoms, box_L, seed );
    
    //build a cell list so we can easily find neighbours of atoms
    cells = buildCellList( atoms, N_atoms, box_L, r_cut);
    
    //give each atom a list of its neighbours, stored inside the ATOM_T as an extensible array.
    buildNeighbourLists( atoms, N_atoms, box_L, r_cut, cells );
    
    U_start = U_LJsys( atoms, N_atoms, box_L, r_cut );
    U_old = U_start;
    printf("U_start %e\n", U_start);
    while (n_step < N_max){
        acc_counter = 0;
        //each 'sweep' move each atom once.
        for(i_atom = 0; i_atom < N_atoms; i_atom ++ ){
             for( i = 0; i < 3; i++){
                 old_x[i] = atoms[i_atom]->position[i];
             }
            //pick a new position a fixed distance away
            propose_move( old_x, new_x, l_step, box_L );
            
            //what is the energy change?
            dU = moveParticle( atoms, N_atoms, box_L, r_cut, cells, i_atom, new_x );
            U_new = U_old + dU;
                        
            // Metropolis Criterion
            if (dU <= 0){
                acc_counter += 1;
            }
            else{
                P_accept = (double)rand()/RAND_MAX; // probability between 0 and 1
                P_pass = exp(- beta * dU);
                //printf("dU is %.3f, probability is %.3f, needs to be %e to pass\n", dU, P_accept, P_pass);
                if (P_accept > P_pass ){ 
                    /* not accepted*/
                    U_new = U_old;
                    moveParticle( atoms, N_atoms, box_L, r_cut, cells, i_atom, old_x );
                }
                else { 
                    acc_counter += 1;
                }
            }
            U_old = U_new; 
        }
        
        x = acc_counter / (double)N_atoms;
        p = p * (n_step / (double)(n_step + 1)) + x / (double)(n_step + 1);
        
        l_step = l_step * ((gamma + p) / ( gamma + 0.5) );
        printf("x %e p %e   lstep %e    dU %e    U_tot %e \n", x, p, l_step, dU, U_new);
            
        n_step += 1;
    }
    free( atoms );
    free( cells );
    
    return( EXIT_SUCCESS );    
}

In [23]:
filter(100, 5.0, 1337, 4.0, 1.0, INFINITY);
filter(100, 5.0, 1337, 4.0, 1.0, 0.5);
filter(100, 5.0, 1337, 4.0, 1.0, 1.0);
filter(100, 5.0, 1337, 4.0, 1.0, 2.0);

U_start 3.832781e+07
x 5.100000e-01 p 5.100000e-01   lstep 1.016667e+00    dU -7.758741e+01    U_tot 1.586079e+04 
x 2.500000e-01 p 3.800000e-01   lstep 8.133333e-01    dU -4.583723e+01    U_tot 4.948021e+03 
x 2.600000e-01 p 3.400000e-01   lstep 5.964444e-01    dU 2.173646e+03    U_tot 2.035524e+03 
x 1.800000e-01 p 3.000000e-01   lstep 3.976296e-01    dU 7.993038e+01    U_tot 1.226749e+03 
x 1.500000e-01 p 2.700000e-01   lstep 2.452049e-01    dU 5.827062e+00    U_tot 8.236845e+02 
x 2.600000e-01 p 2.683333e-01   lstep 1.505286e-01    dU 5.315428e+01    U_tot 4.425595e+02 
x 2.800000e-01 p 2.700000e-01   lstep 9.282596e-02    dU 5.854158e+00    U_tot 3.339451e+02 
x 3.000000e-01 p 2.737500e-01   lstep 5.782284e-02    dU -8.977347e-01    U_tot 1.933061e+02 
x 4.400000e-01 p 2.922222e-01   lstep 3.779900e-02    dU -8.144933e-01    U_tot 1.286900e+02 
x 4.600000e-01 p 3.090000e-01   lstep 2.576632e-02    dU -1.222896e+00    U_tot 8.630131e+01 
x 3.500000e-01 p 3.127273e-01   lstep 1.7724

In [25]:
filter(100, 15.0, 1337, 4.0, 1.0, INFINITY);
filter(100, 15.0, 1337, 4.0, 1.0, 0.5);
filter(100, 15.0, 1337, 4.0, 1.0, 1.0);
filter(100, 15.0, 1337, 4.0, 1.0, 2.0);

U_start 5.038210e+01
x 4.300000e-01 p 4.300000e-01   lstep 8.833333e-01    dU 1.052330e-02    U_tot 1.217717e+01 
x 3.100000e-01 p 3.700000e-01   lstep 6.919444e-01    dU -8.928885e-03    U_tot -9.516101e+00 
x 2.900000e-01 p 3.433333e-01   lstep 5.112701e-01    dU 2.581211e-02    U_tot -1.153573e+01 
x 3.000000e-01 p 3.325000e-01   lstep 3.685405e-01    dU 3.570580e-02    U_tot -1.320498e+01 
x 2.600000e-01 p 3.180000e-01   lstep 2.567499e-01    dU 1.014638e-02    U_tot -1.449207e+01 
x 1.900000e-01 p 2.966667e-01   lstep 1.697402e-01    dU -3.479992e-02    U_tot -1.543405e+01 
x 3.200000e-01 p 3.000000e-01   lstep 1.131601e-01    dU -2.481125e-02    U_tot -1.612047e+01 
x 2.300000e-01 p 2.912500e-01   lstep 7.378984e-02    dU -1.554822e-02    U_tot -1.636672e+01 
x 3.100000e-01 p 2.933333e-01   lstep 4.837334e-02    dU 1.877656e-02    U_tot -1.672909e+01 
x 3.600000e-01 p 3.000000e-01   lstep 3.224889e-02    dU 1.187111e-03    U_tot -1.694591e+01 
x 3.700000e-01 p 3.063636e-01   lste

The filter is kind of bad. The convergence isn't always ensured for a given box size, at different temperatures. The whole process still seems to work, as the total energy is decreasing, thus the algorithm is 'optimizing' the placement in a good way.